Statement 29
Use transfer learning with VGG16 on the Cats and Dogs dataset, freezing the first 4 layers, and train
the classifier and evaluate model performance using a classification report.

#Step 1: Data Preparation

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [ ]:
# Define paths
train_dir = 'cats_and_dogs/train'
validation_dir = 'cats_and_dogs/validation'
test_dir = 'cats_and_dogs/test'

In [ ]:
# Image dimensions
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
batch_size = 32


In [ ]:
# Data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

test_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Step 2: Model Setup with Transfer Learning

In [ ]:
# Load VGG16 base model
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=input_shape
)

# Freeze first 4 layers
for layer in base_model.layers[:4]:
    layer.trainable = False

# Build model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Step 3: Model Training

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Step 4: Evaluation and Classification Report

In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"\nTest Accuracy: {test_acc:.4f}")

# Generate predictions
test_generator.reset()
preds = model.predict(test_generator)
y_pred = np.where(preds > 0.5, 1, 0)

# Classification report
print("\nClassification Report:")
print(classification_report(
    test_generator.classes,
    y_pred,
    target_names=['Cat', 'Dog']
))

# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Curves')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curves')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.tight_layout()
plt.show()

#Transfer Learning Setup:

Uses VGG16 pre-trained on ImageNet

Freezes first 4 convolutional blocks

Adds custom classifier head

#Data Handling:

Proper image augmentation for training

Separate generators for train/val/test

Binary classification setup (cats vs dogs)

#Model Architecture:

VGG16 convolutional base

Flatten layer to transition to dense layers

256-unit dense layer with ReLU activation

50% dropout for regularization

Single output with sigmoid activation

#Evaluation:

Full classification report with precision/recall/F1

Accuracy and loss curves visualization

Test set evaluation